In [125]:
import bioservices
import json
import re

from cache import kegg_cached_reqest
from cache import get_smile_string
from helpers import print_percent_done

In [109]:
# import cache
# import importlib
# importlib.reload(cache)

found JSON cache


<module 'cache' from '/Users/oneoPk/Desktop/flavo/flavoenzymes/cache.py'>

In [57]:
k = bioservices.kegg.KEGG()
parser = bioservices.kegg.KEGGParser()

# This file is WIP still:
### TODO:
- Convert the enzymes to a SMILES strings
- Allow to fetch data from Kegg for ECs that are in Brenda (or other databases)
    - perhaps first, a master list of all FlavoEnzymes can be made, then itterated over
        

### QUESTIONS:
- SMILES string
    - https://www.genome.jp/tools/simcomp/

### Configurables
Edit the folling options before running the scraper

In [163]:
useCaching = True # setting this parameter to False will fetch new data from server, instead of using cache
keywords = ['FAD', 'FMN', 'flavin', 'flavoenzyme']
import_file = "export/kegg.json"
export_file = "export/kegg.json"
poster_child = '1.14.13.2'

### Helper functions

In [37]:
def read_past_data(path=import_file):
    try:
        with open(path) as json_file:
            return json.load(json_file)
    except:
        return {}

In [38]:
def get_ids(keyword):
    results = k.find(database='enzyme', query=keyword)
    results_array = results.split('\n')
    ids_array = [i.split('\t')[0] for i in results_array if (i)]
    return set(ids_array)

In [62]:
def get_ids_for_keywords(keywords):
    all_ids_set = set()
    for keyword in keywords:
        kw_ids = get_ids(keyword);
        all_ids_set = all_ids_set | kw_ids
    return all_ids_set

In [49]:
def kegg_request(id):
    resp = kegg_cached_reqest(request_name=f'kegg_ec_request', ec=id, request_fn=k.get, useCaching=useCaching)
    ec_parse = parser.parse(resp)
    return ec_parse

In [ ]:
def update_compounds(list_of_compounds):
    new_compounds = {}
    pattern = '(.+) \[CPD:(.+)\]'

    for substrate in list_of_compounds:
        if 'CPD' in substrate:
            match = re.findall(pattern, product)[0]
            
            cpd_number = 'cpd:' + match[1]
            print(cpd_number)
            
            name = match[0]
            smiles = cache.get_smile_string(cpd_number)
            
            # updating the entree
            new_substrates[smiles] = {
                'name': name,
                'smiles': smiles,
                'kegg_name': substrate,
            }
            print(f'{cpd_number} ==> smiles: {smiles}\n\n')
        else:
            print(f'Not a valid CPD number for a substrate: {substrate}')

    return new_substrates


In [160]:
def update_substrates(substrates):
    print("="*30)
    print("========= SUBSTRATE ==========")
    print("="*30)

    new_substrates = {}
    pattern = '(.+) \[CPD:(.+)\]'

    for substrate in substrates:
        if 'CPD' in substrate:
            match = re.findall(pattern, substrate)[0]
            
            cpd_number = 'cpd:' + match[1]
            print(cpd_number)
            
            name = match[0]
            smiles = cache.get_smile_string(cpd_number)
            
            # updating the entree
            new_substrates[smiles] = {
                'name': name,
                'smiles': smiles,
                'kegg_name': substrate,
            }
            print(f'{cpd_number} ==> smiles: {smiles}\n\n')
        else:
            print(f'Not a valid CPD number for a substrate: {substrate}')

    return new_substrates

def update_products(products):
    print("="*30)
    print("========== PRODUCT ===========")
    print("="*30)
    new_products = {}
    pattern = '(.+) \[CPD:(.+)\]'
    
    for product in products:
        if 'CPD' in product:
            match = re.findall(pattern, product)[0]
            
            cpd_number = 'cpd:' + match[1]
            print(cpd_number)
            
            name = match[0]
            smiles = cache.get_smile_string(cpd_number)
            
            # updating the entree
            new_products[smiles] = {
                'name': name,
                'smiles': smiles,
                'kegg_name': product,
            }
            print(f'{cpd_number} ==> smiles: {smiles}\n\n')
        else:
            print(f'Not a valid CPD number for a product: {product}')

    return new_products

In [164]:
data = kegg_request(poster_child)
substrates = data['SUBSTRATE']
products = data['PRODUCT']

update_substrates(substrates)
update_products(products)

Getting cached data...
kegg_ec_request?ec=1.14.13.2
========= SUBSTRATE ==========
cpd:C00156
Getting cached data...
smile_string?cpd=cpd:C00156
cpd:C00156 ==> smiles: OC(=O)c1ccc(O)cc1


cpd:C00005
Getting cached data...
smile_string?cpd=cpd:C00005
cpd:C00005 ==> smiles: NC(=O)C1=CN(C=CC1)[C@@H]1O[C@H](COP(O)(=O)OP(O)(=O)OC[C@H]2O[C@H]([C@H](OP(O)(O)=O)[C@@H]2O)n2cnc3c(N)ncnc23)[C@@H](O)[C@H]1O


cpd:C00080
Getting cached data...
smile_string?cpd=cpd:C00080
cpd:C00080 ==> smiles: [1H+]


cpd:C00007
Making a request for new data...
compound_request, cpd: cpd:C00007
cpd:C00007 ==> smiles: [O]


========== PRODUCT ===========
cpd:C00230
Making a request for new data...
compound_request, cpd: cpd:C00230
cpd:C00230 ==> smiles: C(=O)(C1=CC(=C(C=C1)O)O)O


cpd:C00006
Getting cached data...
smile_string?cpd=cpd:C00006
cpd:C00006 ==> smiles: NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP(O)(=O)OP(O)(=O)OC[C@H]2O[C@H]([C@H](OP(O)(O)=O)[C@@H]2O)n2cnc3c(N)ncnc23)[C@@H](O)[C@H]1O


cpd:C00001
Getting cached

{'C(=O)(C1=CC(=C(C=C1)O)O)O': {'name': '3,4-dihydroxybenzoate',
  'smiles': 'C(=O)(C1=CC(=C(C=C1)O)O)O',
  'kegg_name': '3,4-dihydroxybenzoate [CPD:C00230];'},
 'NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP(O)(=O)OP(O)(=O)OC[C@H]2O[C@H]([C@H](OP(O)(O)=O)[C@@H]2O)n2cnc3c(N)ncnc23)[C@@H](O)[C@H]1O': {'name': 'NADP+',
  'smiles': 'NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP(O)(=O)OP(O)(=O)OC[C@H]2O[C@H]([C@H](OP(O)(O)=O)[C@@H]2O)n2cnc3c(N)ncnc23)[C@@H](O)[C@H]1O',
  'kegg_name': 'NADP+ [CPD:C00006];'},
 '[H]O[H]': {'name': 'H2O',
  'smiles': '[H]O[H]',
  'kegg_name': 'H2O [CPD:C00001]'}}

In [161]:
ec_numbers = sorted(kegg.keys())[:10]
for ec in ec_numbers:
    print("="*30)
    print(f">>> ENZ: {ec}")
    print("="*30)

    data = kegg_request(ec)
    substrates = data['SUBSTRATE']
    products = data['PRODUCT']
    
    update_substrates(substrates)
    update_products(products)

>>> ENZ: 1.1.1.1
Getting cached data...
kegg_ec_request?ec=1.1.1.1
========= SUBSTRATE ==========
cpd:C00226
Getting cached data...
smile_string?cpd=cpd:C00226
cpd:C00226 ==> smiles: *C(O)([H])[H]


cpd:C00003
Getting cached data...
smile_string?cpd=cpd:C00003
cpd:C00003 ==> smiles: NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP(O)(=O)OP(O)(=O)OC[C@H]2O[C@H]([C@H](O)[C@@H]2O)n2cnc3c(N)ncnc23)[C@@H](O)[C@H]1O


cpd:C01612
Getting cached data...
smile_string?cpd=cpd:C01612
cpd:C01612 ==> smiles: *C(*)O


========== PRODUCT ===========
cpd:C00071
Getting cached data...
smile_string?cpd=cpd:C00071
cpd:C00071 ==> smiles: [H]C([*])=O


cpd:C00004
Getting cached data...
smile_string?cpd=cpd:C00004
cpd:C00004 ==> smiles: NC(=O)C1=CN(C=CC1)[C@@H]1O[C@H](COP(O)(=O)OP(O)(=O)OC[C@H]2O[C@H]([C@H](O)[C@@H]2O)n2cnc3c(N)ncnc23)[C@@H](O)[C@H]1O


cpd:C00080
Getting cached data...
smile_string?cpd=cpd:C00080
cpd:C00080 ==> smiles: [1H+]


cpd:C01450
Getting cached data...
smile_string?cpd=cpd:C01450
cpd:C01450 ==

In [64]:
def get_all_data(ids, previous_json, verbose=False):    
    data_dict = previous_json
    for index,id in enumerate(ids):
        ec_data = kegg_request(id)
        name = ec_data.get('SYSNAME',id) # if sysname not found, will use ec_number
        ec_number = id.replace('ec:','')
        
        ec_data['KEGG_ID'] = id
        ec_data['EC_NUMBER'] = ec_number
        # WARNING: keys are now EC numbers istead of sysname, but can be changed 
        data_dict[ec_number] = ec_data 
        if verbose: 
            print_percent_done(index=index, length=len(ids))
    return data_dict

In [82]:
def scrape_kegg():
    print(f'Kegg scraping script started...')
    # Reading past results from KEGG
    previous_json = read_past_data()
    print(f'1. Successfully read previous json data, that has total of {len(previous_json)} records')
    prev_ids = {enzyme['KEGG_ID'] for enzyme in list(previous_json.values())}
    
    # Getting IDs of all entries that are missing from past
#     new_ids = get_ids_for_keywords(keywords)
    new_ids = set(brenda_blacklist) - set(benda_not_found)
    
    all_ids = new_ids - prev_ids    
                
    # If new ids have been found, fetch the data
    if len(all_ids) > 0:
        print(f'2. Following potential flavins are missing from past results:')
        [print(f'{ec}', end=" | ") for ec in all_ids]
    
        # Scraping the data
        print(f'\n3. Fetching the data')
        flavins = get_all_data(all_ids, previous_json, verbose=True)
        
        # Writing out the results to the file
        with open(export_file, 'w') as outfile:
            json.dump(flavins, outfile)
        print(f'\nSuccessfully written out {len(all_ids)} results to "{export_file}"')
    else:
        print("[i] Doesn't look like there are any new flavins on KEGG!")

### Running the program

In [ ]:
scrape_kegg()

In [39]:
# prints all the ECs of items without SYSNAME
kegg = read_past_data()
sysnames = [ec for (ec, details) in list(kegg.items()) if not details.get('SYSNAME')]
sysnames

['3.4.22.61',
 '1.16.8.1',
 '1.6.5.11',
 '2.7.7.95',
 '1.6.8.2',
 '1.14.99.40',
 '1.14.99.7',
 '3.4.24.15',
 '1.6.8.1',
 '1.7.99.5',
 '1.13.11.32',
 '2.5.1.77',
 '3.4.24.59',
 '3.4.22.3',
 '1.5.1.29',
 '3.4.24.69',
 '3.4.21.109',
 '3.4.16.4',
 '3.4.21.1',
 '1.14.19.7',
 '1.1.99.15',
 '3.4.22.56']

### Seeing how many brenda flavins are in Kegg

In [27]:
brenda = read_past_data('export/brenda.json')
brenda_ecs = sorted(brenda.keys())
# brenda_flavins = get_all_data(brenda_ecs, previous_json = {}, verbose=True)
benda_not_found=[]
for index,id in enumerate(brenda_ecs):
        try:
            ec_data = kegg_request(id)
        except:
            benda_not_found.append(id)

In [29]:
len(benda_not_found)

781

In [93]:
CPD = k.parse(k.get("cpd:C00156"))

In [94]:
CPD.keys()

dict_keys(['ENTRY', 'NAME', 'FORMULA', 'EXACT_MASS', 'MOL_WEIGHT', 'REACTION', 'PATHWAY', 'MODULE', 'ENZYME', 'DBLINKS', 'ATOM', 'BOND'])

In [95]:
CPD['FORMULA']

'C7H6O3'

In [2]:
import requests

In [22]:

    
target_db = 'uniprot'
source_db = 'compound'
cpd = 'cpd:C00156'
url = f'http://rest.kegg.jp/conv/{target_db}/{source_db}'
resp = requests.get(url)
print(resp.text)

NameError: name 'requests' is not defined

In [25]:
# perhaps we can use openbable or pybel?
# https://pypi.org/project/openbabel/3.0.0/
# to convert to smiles string?

### Converting Kegg entry to SMILES

In [105]:
cpd = 'cpd:C00156'
url = f'https://www.genome.jp/dbget-bin/www_bget?-f+m+compound+{cpd}'
resp = requests.get(url)
print(resp.text)

 
 
 
 10 10  0  0  0  0  0  0  0  0999 V2000
   25.6200  -14.7656    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   25.6137  -13.3697    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   24.4098  -15.4637    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   26.8430  -15.4637    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   24.4098  -12.6779    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
   26.8174  -12.6716    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
   24.4098  -16.8659    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   26.8430  -16.8659    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   25.6200  -17.5831    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   25.6137  -18.9727    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
  1  2  1  0     0  0
  1  3  1  0     0  0
  1  4  2  0     0  0
  2  5  1  0     0  0
  2  6  2  0     0  0
  3  7  2  0     0  0
  4  8  1  0     0  0
  7  9  1  0     0  0
  9 10  1  0     0  0
  8  9  2  0     0  0
M  END



In [34]:
get_smile_string(kegg_entry)

Making a request for new data...
ChEBI_request, ec: CHEBI:16048


TypeError: Object of type 'Entity' is not JSON serializable

In [62]:
# chebi_con = bioservices.ChEBI()

kegg_entry = parser.parse(k.get('cpd:C00156'))
ChEBI_links = kegg_entry['DBLINKS']['ChEBI'].split(' ')
ChEMBL_links = kegg_entry['DBLINKS'].get('ChEMBL', None)
print(ChEBI_links)

# chebi_entry = chebi_con.getCompleteEntity('CHEBI:' + kegg_entry['DBLINKS']['ChEBI'])

# print (chebi_entry.smiles)
# print (chebi_entry.inchi)
# print (chebi_entry.inchiKey)


# >>> from bioservices import *
# >>> ch = ChEBI()
# >>> res = ch.getCompleteEntity("CHEBI:27732")
# >>> res.smiles
# CN1C(=O)N(C)c2ncn(C)c2C1=O


['17879', '30763']


In [63]:
map_kegg_chebi['cpd:C00156']

'chebi:30763'

In [60]:
ChEMBL_links

In [18]:
kegg_entry

{'ENTRY': 'C01847                      Compound',
 'NAME': ['Reduced FMN;', 'FMNH2'],
 'FORMULA': 'C17H23N4O9P',
 'EXACT_MASS': 458.1203,
 'MOL_WEIGHT': 458.3597,
 'REACTION': ['R00025',
  'R04102',
  'R05218',
  'R05705',
  'R05706',
  'R07210',
  'R07664',
  'R08908',
  'R09083',
  'R09313',
  'R09314',
  'R09559',
  'R09798',
  'R09819',
  'R09936',
  'R09937',
  'R10203',
  'R10206',
  'R10551',
  'R11161',
  'R11162',
  'R11163',
  'R11164',
  'R11225',
  'R11404',
  'R11454',
  'R11455',
  'R11456',
  'R11457',
  'R12149',
  'R12538',
  'R12544',
  'R12545',
  'R12546',
  'R12547'],
 'PATHWAY': {'map00740': 'Riboflavin metabolism',
  'map01057': 'Biosynthesis of type II polyketide products',
  'map01100': 'Metabolic pathways'},
 'ENZYME': ['1.1.99.31',
  '1.3.99.24',
  '1.5.1.36',
  '1.5.1.38',
  '1.5.1.39',
  '1.5.1.41',
  '1.5.1.42',
  '1.5.1.-',
  '1.13.11.79',
  '1.13.12.16',
  '1.14.14.3',
  '1.14.14.5',
  '1.14.14.10',
  '1.14.14.12',
  '1.14.14.13',
  '1.14.14.21',
  '1.14